In [1]:
import sys
sys.path.append('/home/ph/LLM2/swift')
sys.path.insert(0,'/home/ph/LLM2/swift')

测试qwen-vl-chat

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType, get_default_template_type,
)
from swift.utils import seed_everything

model_type = ModelType.qwen_vl_chat
template_type = get_default_template_type(model_type)
print(f'template_type: {template_type}')  # template_type: qwen

model, tokenizer = get_model_tokenizer(model_type, model_kwargs={'device_map': 'auto'})

template = get_template(template_type, tokenizer)
seed_everything(42)
query = tokenizer.from_list_format([
    {'image': 'https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg'},
    {'text': '这是什么'},
])
response, history = inference(model, template, query)
print(f'query: {query}')
print(f'response: {response}')
query = '输出击掌的检测框'
response, history = inference(model, template, query, history)
print(f'query: {query}')
print(f'response: {response}')
print(f'history: {history}')
image = tokenizer.draw_bbox_on_latest_picture(response, history)
image.save('output_chat.jpg')

/home/ph/anaconda3/envs/torch2.1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-06 10:09:52,707 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-05-06 10:09:52,709 - modelscope - INFO - Loading ast index from /home/ph/.cache/modelscope/ast_indexer
2024-05-06 10:09:52,745 - modelscope - INFO - Loading done! Current index file version is 1.10.0, with md5 f6e226a8578971e8fd7923bb0e86893b and a total number of 946 components indexed
2024-05-06 10:09:53,355	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
[INFO:swift] Loading the model using model_dir: /home/ph/LLM2/VL/Qwen-VL-Chat
[INFO:swift] Setting torch_dtype: torch.bfloat16


template_type: qwen


Loading checkpoint shards: 100%|██████████| 10/10 [00:04<00:00,  2.20it/s]
[INFO:swift] model.max_model_len: 2048
[INFO:swift] Global seed set to 42


query: Picture 1: <img>https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg</img>
这是什么
response: 图中是一名女子在沙滩上和狗玩耍，旁边的狗是一只拉布拉多犬，它们处于沙滩上。
query: 输出击掌的检测框
response: <ref>击掌</ref><box>(523,513),(584,605)</box>
history: [['Picture 1: <img>https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg</img>\n这是什么', '图中是一名女子在沙滩上和狗玩耍，旁边的狗是一只拉布拉多犬，它们处于沙滩上。'], ['输出击掌的检测框', '<ref>击掌</ref><box>(523,513),(584,605)</box>']]


# sft

测试sft更改认知

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
import sys
sys.path.append('/home/ph/LLM2/swift')
sys.path.insert(0,'/home/ph/LLM2/swift')
from swift.llm import DatasetName, ModelType, SftArguments, sft_main

sft_args = SftArguments(
    model_type=ModelType.qwen_14b_chat,
    # model_id_or_path='/home/ph/LLM/Qwen1.5/Qwen1.5-14B-Chat-GPTQ-Int4',  model_cache_dir='/home/ph/LLM/Qwen1.5/Qwen1.5-14B-Chat-GPTQ-Int4',
    dataset=[DatasetName.sharegpt_gpt4_mini],
    train_dataset_sample=1000,
    logging_steps=5,
    max_length=2048,
    warmup_ratio=0.4,
    output_dir='output',
    lora_target_modules=['ALL'],   # 在所有的linear层(包括qkvo以及mlp)加lora. 这通常是效果最好的.
    self_cognition_sample=500,
    model_name=['猴乐乐', 'houlele'],
    model_author=['同元智算', 'Tongyuan intelligence calculation'])
output = sft_main(sft_args)
best_model_checkpoint = output['best_model_checkpoint']
print(f'best_model_checkpoint: {best_model_checkpoint}')

/home/ph/anaconda3/envs/torch2.1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-06 10:31:31,921 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-05-06 10:31:31,927 - modelscope - INFO - Loading ast index from /home/ph/.cache/modelscope/ast_indexer
2024-05-06 10:31:31,967 - modelscope - INFO - Loading done! Current index file version is 1.10.0, with md5 f6e226a8578971e8fd7923bb0e86893b and a total number of 946 components indexed
2024-05-06 10:31:32,563	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
[INFO:swift] Setting template_type: qwen
[INFO:swift] Setting args.lazy_tokenize: False
[INFO:swift] output_dir: /home/ph/LLM2/swift/tests/llm/output/qwen-14b-chat/v1-20240506-103132
[INFO:swift] 

device_count: 2
rank: -1, local_rank: -1, world_size: 1, local_world_size: 1


Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 15/15 [00:07<00:00,  2.08it/s]
[INFO:swift] registered_causal_mask to cuda
[INFO:swift] model.max_model_len: 2048
[INFO:swift] model_config: QWenConfig {
  "_name_or_path": "/home/ph/LLM/Qwen-14B-main/Qwen-14B-Chat",
  "architectures": [
    "QWenLMHeadModel"
  ],
  "attn_dropout_prob": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_qwen.QWenConfig",
    "AutoModelForCausalLM": "modeling_qwen.QWenLMHeadModel"
  },
  "bf16": true,
  "emb_dropout_prob": 0.0,
  "fp16": false,
  "fp32": false,
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 27392,
  "kv_channels": 128,
  "layer_norm_epsilon": 1e-06,
  "max_position_embeddings": 8192,
  "model_type": "qwen",
  "no_bias": true,
  "num_attention_heads": 40,
  "num_hidden_layers": 40,
  "onnx_safe": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 1.0,
  "scale_attn_weights": true,
  "seq_length": 2048,
  "tie_wo

TypeError: __init__() got an unexpected keyword argument 'use_seedable_sampler'